In [1]:
class NewsArticle:
    """This is a news article that has a title sentiment and a number of shares"""
    def __init__(
        self,
        news_article_id,
        source_id,
        source_preference,
        sentiment,
        num_shares,
        fake
    ):
        self.news_article_id = news_article_id
        self.source_id = source_id
        self.source_preference=source_preference
        self.sentiment=sentiment #value between (-5,-1) and (1,5)
        self.num_shares = num_shares # any value >=0
        self.fake=fake #either real=0 or fake=1
        
class User():
    # THIS COULD BE THE USER
    
    #CONSTRUCTOR -- REPLACE WITH USER ATTRIBUTES
    def __init__(
        self,
        unique_id,
        news_spread_chance,
        preference,
        articles,
    ):
        super().__init__(unique_id, model)

        self.news_spread_chance = news_spread_chance
        self.preference = preference
        self.articles = articles #set of article ids

    def try_to_create_news(self):
        news_creation_percentage = 0.3
        if self.random.random()<news_creation_percentage:
            #check if id exists
            news_article_id = randint(0,100000)
            while news_article_id in list(self.model.news.keys()):
                news_article_id = randint(0,100000)
            #create news
            is_fake = generate_weighted_percentage(0.3)
            if is_fake:
                #make appropriate sentiment
                senti_score = round(np.random.normal(2, 0.5),2)
                if senti_score > 5:
                    senti_score = 5
                senti_score = senti_score*positive_or_negative()
                #make appropriate number of shares
                original_shares = int(np.random.exponential(10))
            else:
                #make appropriate sentiment
                senti_score = round(np.random.normal(2, 0.5),2)
                if senti_score > 5:
                    senti_score = 5
                elif senti_score < 1:
                    senti_score = 1
                senti_score = senti_score*positive_or_negative()
                #make appropriate number of shares
                original_shares = int(np.random.exponential(50))
                
            article = NewsArticle(news_article_id=news_article_id,
                                    source_id=self.unique_id, 
                                    source_preference=self.preference,
                                    sentiment=senti_score,
                                    num_shares= original_shares,
                                    fake=is_fake)
            self.model.news[news_article_id] = article
            self.articles.add(article)
#             print(self.unique_id,'added',news_article_id)
    
    def try_to_spread_news(self):
        """
        Here we will look at the most important features and decide if we want to spread news or not
        """
        if not self.articles:
            return
        else:
            #1-get the neighbors
            neighbors_nodes = self.model.grid.get_neighbors(self.pos, include_center=False)
            susceptible_neighbors = [
                agent
                for agent in self.model.grid.get_cell_list_contents(neighbors_nodes)
            ]
            #2-for each article, try to spread the news
            for article in self.articles:
                for neighbor in susceptible_neighbors:
                    #high source credibility SO SIMILAR THEY ARE LESS THAN RANDOM PROBABILITY
                    if abs(self.preference - neighbor.preference) < self.random.random():
                        #high measure of sentiment GREATER THAN RANDOM PROBABILITY
                        if abs(article.sentiment)/5 > random.random():
                            neighbor.articles.add(article)
#                             print('spreading news')
                        
    def step(self):
        self.try_to_create_news()
        self.try_to_spread_news()